In [1]:
import numpy as np
import pandas as pd

In [2]:
# Assign data
RND = [[24,35,65,81,54],[3,87,27,73,70],[47,45,48,17,9],[42,87,26,36,40],[7,63,19,88,94]]
RNL = [[0,0,0,0,5],[0,0,0,0,0],[0,0,0,0,3],[0,0,0,0,4],[0,0,0,0,8]]
M,N,C,fill,demand = 11,5,5,8,[]
shortage = np.zeros((N,C))
order = np.zeros((N,C))
inv_beg = np.zeros((N,C))
inv_end = np.zeros((N,C))
lead_time = np.zeros((N,C))
shortage_times = 0

In [3]:
#Get the Random Digit Assignment for Demand
def Demand(RND):
    if RND in range(1,11):
        return 0
    elif RND in range(11,36):
        return 1
    elif RND in range(36,71):
        return 2
    elif RND in range(71,92):
        return 3
    elif RND in range(92,100):
        return 4

In [4]:
#Get the Random Digit Assignment for lead time
def leadtime(RNL):
    if RNL in range(1,7):
        return 1
    elif RNL in range(7,10):
        return 2
    elif RNL == 0:
        return 3

In [5]:
for i in range(0,C): #cycle 
    for j in range(0,N):
        
        # Calculate Demand
        D = Demand(RND[i][j])
        demand.append(D)
        
        # Initial state
        if i == 0 and j == 0:
            inv_beg[i][j] = 3
            lead_time[i][j] = 2
        
        # Calculte Inventory begin and reduce lead time
        elif i > 0 and j == 0:         # بدايه cycly
            lead_time[i][j] = lead_time[i-1][4] - 1 #  lead time يقل يوم 
            inv_beg[i][j] = inv_end[i-1][4]  

        else:
            lead_time[i][j] = lead_time[i][j-1] - 1   # لو مش بدايه cycly 

            if int(lead_time[i][j]) == 0:   # لما lead time بيساوى صفر
                                            # بعمل refill
                inv_beg[i][j] = inv_end[i][j-1] + fill - shortage[i][j-1]

            else:
                inv_beg[i][j] = inv_end[i][j-1]   # لما lead time مش بيساوى صفر
            
        # Calculate shortage
        if D > inv_beg[i][j]:
            shortage[i][j] = shortage[i][j-1] + (D - inv_beg[i][j])
            shortage_times += 1
            inv_beg[i][j] = 0  # عشان متبقاش سالب
        
        # Calculate Inventory End
        else:
            inv_end[i][j] = inv_beg[i][j] - D
        
        # Make an order and calculate Lead time
        if j == 4:       # اخر يوم ف cycle
            order[i][j] = M - inv_end[i][j] + shortage[i][j]
            lead_time[i][j] = leadtime(RNL[i][j]) + 1
            fill = order[i][j]
        
        # Edit lead time values
        if lead_time[i][j] < 0 :
            lead_time[i][j] = 0

In [6]:
# Generate Days and Cycles
Cycles,Days = [],[]
for i in range(1,6):
    for k in range(1,6):
        Cycles.append(i)
for j in range(1,26):
    Days.append(j)

In [7]:
# convert 2D Numpy array to 1D Numpy array
sum_inv_end = sum(inv_end)
inv_beg = np.ravel(inv_beg)
inv_end = np.ravel(inv_end)
RND = np.ravel(RND)
RNL = np.ravel(RNL)
shortage = np.ravel(shortage)
order = np.ravel(order)
lead_time = np.ravel(lead_time)

In [8]:
# Create Pandas DataFrame

data = {
    "Cycle" : Cycles,
    "Days" : Days,
    "INV_B" : inv_beg,
    "RND" : RND,
    "Demand" : demand,
    "INV_End" : inv_end,
    "Shortage" : shortage,
    "Order" : order,
    "RNL" : RNL,
    "Lead" : lead_time
}
series = pd.DataFrame(data)
print(series)

    Cycle  Days  INV_B  RND  Demand  INV_End  Shortage  Order  RNL  Lead
0       1     1    3.0   24       1      2.0       0.0    0.0    0   2.0
1       1     2    2.0   35       1      1.0       0.0    0.0    0   1.0
2       1     3    9.0   65       2      7.0       0.0    0.0    0   0.0
3       1     4    7.0   81       3      4.0       0.0    0.0    0   0.0
4       1     5    4.0   54       2      2.0       0.0    9.0    5   2.0
5       2     6    2.0    3       0      2.0       0.0    0.0    0   1.0
6       2     7   11.0   87       3      8.0       0.0    0.0    0   0.0
7       2     8    8.0   27       1      7.0       0.0    0.0    0   0.0
8       2     9    7.0   73       3      4.0       0.0    0.0    0   0.0
9       2    10    4.0   70       2      2.0       0.0    9.0    0   4.0
10      3    11    2.0   47       2      0.0       0.0    0.0    0   3.0
11      3    12    0.0   45       2      0.0       2.0    0.0    0   2.0
12      3    13    0.0   48       2      0.0       

In [9]:
# Calculate Inventory End Average and shortage times
print("Average End of Inentory =",sum(sum_inv_end)/(N*C),"units.")
print("The number of shortage times = {} times out of {} days.".format(shortage_times,(N*C)))

Average End of Inentory = 3.52 units.
The number of shortage times = 2 times out of 25 days.
